# * Interim : Target & Baseline %EBITDA
    H level only

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
# import xlrd

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 20)

## ETL Process...

### Step 1 : Import Data Source

In [2]:
''' Rawdata '''

# Target & Baseline_EBITDA.xlsx
src_file = '../../data/interim/Target & Baseline_EBITDA.xlsx'
src_sheet = 'Rawdata'
src_skiprows = 2
src_usecols = 'B:L'
src_df = pd.read_excel(src_file, sheet_name=src_sheet, skiprows=src_skiprows, usecols=src_usecols, index_col=None) 
# src_df = src_df.replace(np.nan, None)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df.tail(3)


src_df : 650 rows, 11 columns


,TM_KEY_MTH,COMP_CD,METRIC_CD,METRIC_NAME,METRIC_VALUE,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REMARK
647,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,90Z,SONGKHLA,NaN
648,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,93X,"TRANG, SATUN, PHATTHALUNG",NaN
649,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",NaN


In [29]:
''' Master Data '''

'''
# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
# Create H level
mooc_h_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['HOP_HINT']!='True Corp')]
mooc_h_df = mooc_h_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT']].drop_duplicates()
mooc_h_df.dropna(how='all', inplace=True)
mooc_h_df['H_AREA_KEY'] = mooc_h_df['ORGID_H'].apply(lambda x: x if re.search(r'[a-z]', x, re.I) else int(x))
# mooc_h_df.tail(3)
'''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)
dt_df.tail(3)

,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TM_KEY_MTH,MONTH_SHORT,TM_KEY_YR
1093,20251229,31,2026001,202512,Dec,2025
1094,20251230,31,2026001,202512,Dec,2025
1095,20251231,31,2026001,202512,Dec,2025


In [23]:
''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'METRIC_VALUE': 'mean', 'TM_KEY_MTH': 'nunique', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_MTH': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df

VERSION COMP_CD METRIC_CD METRIC_NAME TM_KEY_MTH         METRIC_VALUE  \
                                               min     max         mean   
0       B     ALL  CAP00006     %EBITDA     202401  202405    41.978431   
1       T     ALL  CAP00006     %EBITDA     202401  202405    61.055215   

  AREA_TYPE AREA_CD  
    nunique nunique  
0         1      65  
1         1      65

### Step 2 : Aggregate Data

In [41]:
'''
    CAP00006	%EBITDA
'''

''' Filter '''
raw_df = src_df


''' Data Test '''
# raw_df = raw_df.loc[raw_df['REMARK'].isna()]
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH']=='202405']
# raw_df = raw_df.loc[raw_df['AREA_CD']=='003']
raw_df = raw_df.reset_index(drop=True)

print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df.tail(3)


raw_df : 650 rows, 11 columns


,TM_KEY_MTH,COMP_CD,METRIC_CD,METRIC_NAME,METRIC_VALUE,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REMARK
647,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,90Z,SONGKHLA,NaN
648,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,93X,"TRANG, SATUN, PHATTHALUNG",NaN
649,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",NaN


In [42]:
''' Merge Data '''

# # Join Area
# merge_df1 = pd.merge(raw_df, mooc_h_df, how='left', on='H_AREA_KEY')

# Join Period
merge_df1 = pd.merge(raw_df, dt_df, how='left', on='TM_KEY_MTH')

# Prep Aggregate
prep_agg_df = merge_df1
prep_agg_df.rename(columns={'METRIC_VALUE': 'MTH_VALUE'}, inplace=True)
prep_agg_df['DAY_VALUE'] = prep_agg_df['MTH_VALUE']

print(f'\nprep_agg_df : {prep_agg_df.shape[0]} rows, {prep_agg_df.shape[1]} columns')
prep_agg_df.tail(3)


prep_agg_df : 19760 rows, 17 columns


,TM_KEY_MTH,COMP_CD,METRIC_CD,METRIC_NAME,MTH_VALUE,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REMARK,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,MONTH_SHORT,TM_KEY_YR,DAY_VALUE
19757,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",NaN,20240529,31,2024022,May,2024,59.846
19758,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",NaN,20240530,31,2024022,May,2024,59.846
19759,202405,ALL,CAP00006,%EBITDA,59.846,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",NaN,20240531,31,2024022,May,2024,59.846


In [44]:
''' Aggregate H level only '''

agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

last_agg_all_df = prep_agg_df.loc[:, agg_cols]

print(f'\nlast_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns')
last_agg_all_df.tail(3)


last_agg_all_df : 19760 rows, 14 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE
19757,2024,202405,2024022,20240529,CAP00006,%EBITDA,ALL,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",59.846,59.846
19758,2024,202405,2024022,20240530,CAP00006,%EBITDA,ALL,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",59.846,59.846
19759,2024,202405,2024022,20240531,CAP00006,%EBITDA,ALL,T,3,H,96X,"PATTANI, YALA, NARATHIWAT",59.846,59.846
